# JVS corpus download

In [ ]:
!pip install -U gdown
!gdown https://drive.google.com/u/0/uc?id=19oAw8wWn3Y7z6CKChRdAyGOB9yupL_Xt
!unzip jvs_ver1.zip

In [ ]:
import pathlib

kaldi_all_path = pathlib.Path("./data/jvs")

In [ ]:
import glob
import os
import shutil

# ディレクトリを作成
if kaldi_all_path.exists():
  shutil.rmtree(str(kaldi_all_path))
kaldi_all_path.mkdir(exist_ok=True, parents=True)

result_dict = {}

jvs_base= "jvs_ver1"
not_exist_count = 0

for utt_dir in glob.glob(f"{jvs_base}/*"):
  if "/jvs" not in utt_dir:
    continue
  spk_id = utt_dir.split("/")[-1]


  dir_types = ["falset10","nonpara30","parallel100","whisper10"]

  # falset10
  for dir_type in dir_types:
    with open(utt_dir+f"/{dir_type}/transcripts_utf8.txt") as f:
      lines = f.readlines()
    for line in lines:
      id_text = line.split(":")
      id = id_text[0]
      text = id_text[1]
      wav_path = os.path.join(jvs_base,spk_id,dir_type,"wav24kHz16bit",id+".wav")
      if not pathlib.Path(wav_path).exists():
        not_exist_count +=1
        continue
      if spk_id not in result_dict:
        result_dict[spk_id] = []
      result_dict[spk_id].append({"utt_id":id, "text":text,"wav":wav_path})

print(f"nonpara : {not_exist_count}")
with open(kaldi_all_path/"wav.scp","w") as wav_scp, open(kaldi_all_path/"text", "w") as text, open(kaldi_all_path/"utt2spk","w") as utt2spk:
  for spk_id, value in result_dict.items():
    for v in value:
      text.write(f"{spk_id}_{v['utt_id']} {v['text']}")
      wav_scp.write(f"{spk_id}_{v['utt_id']} {v['wav']}\n")
      utt2spk.write(f"{spk_id}_{v['utt_id']} {spk_id}\n")

nonpara : 2018


In [ ]:
# make spk2utt

dir_list = ["jvs_kaldi/all" ]


for dir in dir_list:
    spk2utt_dict = {}
    utt2spk_path = os.path.join(dir,"utt2spk")
    with open(utt2spk_path) as f:
        lines = f.readlines()
    for line in lines:
        utt_spk = line.rstrip().split(" ")
        if utt_spk[1] not in spk2utt_dict:
            spk2utt_dict[utt_spk[1]] =[]
        spk2utt_dict[utt_spk[1]].append(utt_spk[0])
    with open(os.path.join(dir, "spk2utt"), "w") as spk2utt: 
        for spk , utt in spk2utt_dict.items():
            spk2utt.write(f"{spk} {' '.join(utt)}\n")


In [18]:
! bash sort.sh jvs_kaldi/all jvs_kaldi/all